<a href="https://colab.research.google.com/github/hsyvy/Expert-Python-Programming_Second-Edition/blob/master/aptos_binary_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!mkdir /content/drive/My\ Drive/model_logs/aptos_binary/


In [0]:
!ls

sample_data


In [0]:
!mkdir model_logs

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json
Saving se_resnet.py to se_resnet.py
Saving cn_se_resnet.py to cn_se_resnet.py


{'cn_se_resnet.py': b'import torch\nimport torch.nn as nn\n\n\ndef conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):\n    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,\n                     padding=dilation, groups=groups, bias=False, dilation=dilation)\n\n\ndef conv1x1(in_planes, out_planes, stride=1):\n    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)\n\n\nclass SeBlock(nn.Module):\n\n    def __init__(self, inplanes, reduction=1):\n        super(SeBlock, self).__init__()\n\n        self.avg_pool = nn.AdaptiveAvgPool2d(1)\n        self.fc1 = nn.Conv2d(inplanes, inplanes // reduction, kernel_size=1)\n        self.relu = nn.ReLU(inplace=True)\n        self.fc2 = nn.Conv2d(inplanes // reduction, inplanes, kernel_size=1)\n        self.sigmoid = nn.Sigmoid()\n\n    def forward(self, x):\n        out = self.avg_pool(x)\n        out = self.fc1(out)\n        out = self.relu(out)\n        out = self.fc2(out)\n        out

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c aptos2019-blindness-detection

In [0]:
!mkdir train
!mkdir test
!unzip train_images.zip -d train
!unzip test_images.zip -d test

In [0]:
import pandas as pd
train = pd.read_csv('train.csv')
train['score'] = (train.diagnosis > 0)*1
score1 = train[train.score == 1]
score0 = train[train.score == 0]
score1 = score1.sample(frac=1).reset_index(drop=True)
score0 = score0.sample(frac=1).reset_index(drop=True)
val = pd.merge(score1[:350], score0[:350], how='outer')
train_df = pd.merge(score1[350:], score0[350:], how='outer')
val = val.sample(frac=1).reset_index(drop=True)
train_df = train_df.sample(frac=1).reset_index(drop=True)
train_df.drop('diagnosis', axis =1, inplace=True)
val.drop('diagnosis', axis = 1,  inplace=True)
train_df.to_csv('train_binary.csv', index=False)
val.to_csv('val_binary.csv', index=False)

In [0]:
!ls

cn_se_resnet.py  sample_submission.csv	test_images.zip   train_images.zip
kaggle.json	 se_resnet.py		train		  val_binary.csv
model_logs	 test			train_binary.csv
sample_data	 test.csv		train.csv


In [0]:
from cn_se_resnet import cn_se_resnet50
model = cn_se_resnet50().cuda()
print(model)

In [0]:
import os
from glob import glob
import numpy as np
from PIL import Image
import imageio
import matplotlib.pyplot as plt 
import pandas as pd
import torch
from torch.utils import data
import torch.nn.functional as F
import torch.nn as nn
# from torchvision.models import resnet50
# from se_resnet import se_resnext50_32x4d
from cn_se_resnet import cn_se_resnet50

from torchsummary import summary
from skimage import io, transform
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import time

%matplotlib inline

##############################################################################

IMG_SIZE = 512
class AptosBlindnessDetectionDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform = None):
        self.image_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.image_df)
    
    def __getitem__(self, idx):
        folder = self.image_df.id_code.values[idx] + '.png'
        path = os.path.join(self.root_dir, folder)
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
            
        label = self.image_df.score.values[idx]
        
            
        return image, label
    
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation((0, 360)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_data = AptosBlindnessDetectionDataset('train_binary.csv', 'train/', transform=transform_train)
val_data = AptosBlindnessDetectionDataset('val_binary.csv', 'train/', transform=transform_test)



base_model = cn_se_resnet50().cuda()

class Model(nn.Module):
    def __init__(self, base_model):
        super(Model, self).__init__()
        self.initial = base_model
        self.fc = nn.Linear(1000, 2)
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        out = self.initial(x)
        out = F.relu(out)
        out = self.fc(out)
        out = self.softmax(out)
        
        return out
        
model = Model(base_model).cuda()
      

N_EPOCHS = 50
BATCH_SIZE = 12
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=0)
criterion = nn.CrossEntropyLoss()






def train():
    
    model.train()
    total_loss = 0.0
    total_corrects = 0.0

    trainLoader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    for i, data in enumerate(trainLoader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        
        inputs = torch.reshape(inputs, (-1, 1, 3, 512, 512))
        

        output = model(inputs)
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        _, preds = torch.max(output, 1)
        total_corrects += torch.sum(preds == labels.data)
        total_loss += loss.item()
        
        
    total_avg_loss = total_loss/((i+1)*BATCH_SIZE)
    train_acc = total_corrects.float()/((i+1)*BATCH_SIZE)
    print(f'train loss: {total_avg_loss}, train acc: {train_acc}')


def val():

    model.eval()

    total_loss = 0.0
    total_corrects = 0.0

    valLoader = DataLoader(val_data, batch_size=BATCH_SIZE)

    for i, data in enumerate(valLoader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.reshape(inputs, (-1, 1, 3, 512, 512))
        
        with torch.no_grad():
            output = model(inputs)
            loss = criterion(output, labels)
            _, preds = torch.max(output, 1)
            
        total_corrects += torch.sum(preds == labels.data)
        total_loss += loss.item()

    total_avg_loss = total_loss/((i+1)*BATCH_SIZE)
    val_acc = total_corrects.float()/((i+1)*BATCH_SIZE)
    print(f'val loss: {total_avg_loss}, val acc: {val_acc}')

    return total_avg_loss






best_avg_loss = 10
for epoch in range(N_EPOCHS):
    print('*'*50)
    print(f'_________EPOCH {epoch}__________')
    t0 = time.time()
    train()
    val_loss = val()
    print(f'time: {(time.time()-t0)/60}')

    if val_loss < best_avg_loss:
        torch.save(model.state_dict(), f'model_logs/resnext_{epoch}_{val_loss}')
        print("model is saved!!!")
        best_avg_loss = val_loss





**************************************************
_________EPOCH 0__________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


train loss: 0.04634144039530503, train acc: 0.7621456980705261
val loss: 0.03831630789459088, val acc: 0.8347457647323608
time: 13.372220007578532
model is saved!!!
**************************************************
_________EPOCH 1__________
train loss: 0.03809547623400746, train acc: 0.8539136052131653
val loss: 0.03658105626618121, val acc: 0.8587570786476135
time: 13.419166572888692
model is saved!!!
**************************************************
_________EPOCH 2__________
train loss: 0.036830118149720065, train acc: 0.8636976480484009
val loss: 0.03612709579999838, val acc: 0.8587570786476135
time: 13.47375998099645
model is saved!!!
**************************************************
_________EPOCH 3__________
train loss: 0.03566151609023412, train acc: 0.8836032152175903
val loss: 0.03364953095630064, val acc: 0.9067796468734741
time: 13.496878163019817
model is saved!!!
**************************************************
_________EPOCH 4__________
train loss: 0.03558873231353

KeyboardInterrupt: ignored

In [0]:
IMG_SIZE = 512
class AptosBlindnessDetectionDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform = None):
        self.image_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.image_df)
    
    def __getitem__(self, idx):
        folder = self.image_df.id_code.values[idx] + '.png'
        path = os.path.join(self.root_dir, folder)
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
            
        label = self.image_df.score.values[idx]
        
            
        return image, label
    
transform_pipline = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation((0, 360)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_data = AptosBlindnessDetectionDataset('train_binary.csv', 'train/', transform=transform_pipline)
val_data = AptosBlindnessDetectionDataset('val_binary.csv', 'train/', transform=transform_pipline)

In [0]:
base_model = resnet50().cuda()

class Model(nn.Module):
    def __init__(self, base_model):
        super(Model, self).__init__()
        self.initial = base_model
        self.fc = nn.Linear(1000, 2)
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        out = self.initial(x)
        out = F.relu(out)
        out = self.fc(out)
        out = self.softmax(out)
        
        return out
        
model = Model(base_model).cuda()
      

In [0]:
N_EPOCHS = 15
BATCH_SIZE = 24
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.001)
criterion = nn.CrossEntropyLoss()

In [0]:

def train():
    
    model.train()
    total_loss = 0.0
    total_corrects = 0.0

    trainLoader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    for i, data in enumerate(trainLoader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()

        output = model(inputs)
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        _, preds = torch.max(output, 1)
        total_corrects += torch.sum(preds == labels.data)
        print(total_corrects.float())
        total_loss += loss.item()
        
        
    total_avg_loss = total_loss/((i+1)*BATCH_SIZE)
    train_acc = total_corrects.float()/((i+1)*BATCH_SIZE)
    print(f'train loss: {total_avg_loss}, train acc: {train_acc}')


def val():

    model.eval()

    total_loss = 0.0
    total_corrects = 0.0

    valLoader = DataLoader(val_data, batch_size=BATCH_SIZE)

    for i, data in enumerate(valLoader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        
        
        with torch.no_grad():
            output = model(inputs)
            loss = criterion(output, labels)
            _, preds = torch.max(output, 1)
            
        total_corrects += torch.sum(preds == labels.data)
        total_loss += loss.item()

    total_avg_loss = total_loss/((i+1)*BATCH_SIZE)
    val_acc = total_corrects.float()/((i+1)*BATCH_SIZE)
    print(f'val loss: {total_avg_loss}, val acc: {val_acc}')

    return total_avg_loss


In [0]:
best_avg_loss = 10
for epoch in range(N_EPOCHS):
    print('*'*50)
    print(f'_________EPOCH {epoch}__________')
    t0 = time.time()
    train()
    val_loss = val()
    print(f'time: {(time.time()-t0)/60}')

    if val_loss < best_avg_loss:
        torch.save(model.state_dict(), f'/content/drive/My\ Drive/model_logs/aptos_binary/model_{epoch}_{val_loss}')
        print("model is saved!!!")
        best_avg_loss = val_loss


In [0]:
while(True):
    pass

In [0]:
trainLoader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
for i, data in enumerate(trainLoader, 0):
    inputs, labels = data
    inputs, labels = inputs.cuda(), labels.cuda()
    print(inputs.shape)
    print(labels.shape)
    


In [0]:
!nvidia-smi

Mon Jul 22 15:19:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!ls